In [1]:
import re
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
def pre_processing(sentence):
    return remove_stopwords(tokenize_sentence(remove_punctuation(sentence)))

# Remove punctuation from a list of words
def remove_punctuation(sentence):
    return re.sub(r'[^\w\s]', '', sentence)

# Remove stopwords from a list of words
def remove_stopwords(words_list):
    stopwords = open("stop_words_FULL.txt", "r")
    stopwords_list = []
    for word in stopwords:
        stopwords_list.append(word.replace('\n', ''))
    stopwords.close()
    return [value.lower() for value in words_list if value.lower() not in stopwords_list]

# Tokenize the input sentence and also lemmatize its words
def tokenize_sentence(sentence):
    words_list = []
    lmtzr = WordNetLemmatizer()
    for tag in nltk.pos_tag(word_tokenize(sentence)):
        if (tag[1][:2] == "NN"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.NOUN))
        elif (tag[1][:2] == "VB"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.VERB))
        elif (tag[1][:2] == "RB"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.ADV))
        elif (tag[1][:2] == "JJ"):
            words_list.append(lmtzr.lemmatize(tag[0], pos=wn.ADJ))
    return words_list

# Union of the pre-processed words of the definitions and terms from the examples in WN for a sense.
def get_signature(sense):
    signature = []
    for word in tokenize_sentence(sense.definition()):  # definition tokenization
        signature.append(word)
    for example in sense.examples():  # example tokenization
        for word in tokenize_sentence(example):
            # Merge definition and examples
            signature.append(word)
    return signature

Read corpus file. It return a list of list of documents and words for each document.

In [3]:
def read_corpus(txt_file):
    
    with open(txt_file, encoding='utf-8') as file:
        # for each doc create list of pre-processed words in that doc (list of lists)
        documents_words = []

        for line in file:
            if "<doc" in line:  # tag for new doc
                document_words = []  # list of words that will be part of the document
                while True:
                    next_line = file.readline()  # read next line

                    # remove unuseful tags
                    next_line_proc = next_line.replace("<p> ", "").replace("</p>\n", "").replace("/p", "")

                    if "</doc>" in next_line:
                        break
                    # pre-processing steps
                    sentence_words = pre_processing(next_line_proc)
                    document_words.extend(sentence_words)
                documents_words.append(document_words)
        file.close()

    print("Documents number: ", len(documents_words))
    
    return documents_words

Topic modelling using gensim

In [4]:
from gensim import corpora, models

topic_num = 10
topic_words_num = 5

def topic_modelling(documents_words):
    
    # Create a dict with integer keys for all words
    dictionary_LDA = corpora.Dictionary(documents_words)

    # delete all terms that do NOT appear in at least 3 documents.
    # delete all terms that appear in more than 60% of documents (see filter_extremes official doc).
    dictionary_LDA.filter_extremes(no_below=3, no_above=0.6)

    # Converts each document into a list of BoW (list of (id_term, term_frequency) for each term in doc)
    corpus_idbow_freq = [dictionary_LDA.doc2bow(document_words) for document_words in documents_words]
    
    # https://radimrehurek.com/gensim/models/ldamodel.html
    lda_model = models.LdaModel(corpus_idbow_freq, num_topics=topic_num, \
                                id2word=dictionary_LDA, \
                                passes=3, alpha=[0.01] * topic_num, \
                                eta=[0.01] * len(dictionary_LDA.keys()))
    
    return lda_model, corpus_idbow_freq

Show topics.

In [5]:
documents_words = read_corpus("travelling.txt")

model, corpus_idbow_freq = topic_modelling(documents_words)

topics = {'Topic ' + str(i): [(token, round(score, 3)) for token, score in model.show_topic(i, topn=topic_words_num)] for i in range(0, model.num_topics)}
for key, value in topics.items():
    print(key,":", topics[key])

Documents number:  100
Topic 0 : [('word', 0.029), ('travel', 0.027), ('learn', 0.023), ('language', 0.021), ('example', 0.013)]
Topic 1 : [('lesson', 0.088), ('website', 0.023), ('pronunciation', 0.018), ('students', 0.016), ('life', 0.016)]
Topic 2 : [('students', 0.069), ('exam', 0.035), ('book', 0.027), ('sb', 0.027), ('speak', 0.025)]
Topic 3 : [('happen', 0.015), ('word', 0.014), ('clause', 0.014), ('example', 0.011), ('tense', 0.011)]
Topic 4 : [('book', 0.025), ('hotel', 0.023), ('holiday', 0.015), ('beach', 0.014), ('lesson', 0.012)]
Topic 5 : [('lesson', 0.039), ('student', 0.017), ('clause', 0.013), ('audio', 0.013), ('dailystep', 0.01)]
Topic 6 : [('travel', 0.023), ('hotel', 0.021), ('place', 0.018), ('visit', 0.015), ('stay', 0.015)]
Topic 7 : [('clause', 0.033), ('article', 0.018), ('level', 0.018), ('example', 0.017), ('video', 0.016)]
Topic 8 : [('clause', 0.028), ('conditionals', 0.015), ('situation', 0.015), ('result', 0.015), ('happen', 0.014)]
Topic 9 : [('students

Show topics for each document.

In [6]:
print ("Documents topic list")
for i in range (0, len(corpus_idbow_freq)):
    print ("Doc", i, ":", model[corpus_idbow_freq[i]])

Documents topic list
Doc 0 : [(0, 0.41655), (4, 0.11179409), (5, 0.37715492), (6, 0.09350246)]
Doc 1 : [(0, 0.31275022), (2, 0.31053978), (4, 0.37648568)]
Doc 2 : [(0, 0.35156614), (8, 0.6465774)]
Doc 3 : [(2, 0.14442235), (6, 0.8551233)]
Doc 4 : [(5, 0.99982005)]
Doc 5 : [(0, 0.99993885)]
Doc 6 : [(4, 0.999012)]
Doc 7 : [(5, 0.2981226), (8, 0.7012895)]
Doc 8 : [(0, 0.18161604), (3, 0.083086155), (6, 0.69475526), (9, 0.040486548)]
Doc 9 : [(0, 0.11459823), (8, 0.8851316)]
Doc 10 : [(3, 0.56199676), (7, 0.041094378), (8, 0.39681107)]
Doc 11 : [(3, 0.99972486)]
Doc 12 : [(4, 0.052934803), (5, 0.8475884), (8, 0.09880422)]
Doc 13 : [(5, 0.7594556), (7, 0.24025038)]
Doc 14 : [(1, 0.999503)]
Doc 15 : [(5, 0.03097208), (8, 0.53964955), (9, 0.42926398)]
Doc 16 : [(3, 0.4124174), (7, 0.26083127), (8, 0.326196)]
Doc 17 : [(6, 0.680636), (8, 0.25303105), (9, 0.06613853)]
Doc 18 : [(5, 0.21412954), (8, 0.785572)]
Doc 19 : [(7, 0.017388549), (8, 0.9822953)]
Doc 20 : [(8, 0.9998232)]
Doc 21 : [(8, 0